# Importación de librerías

In [2]:
from DatasetLoader import DatasetLoader
from MethodDeepGCNResNet import MethodDeepGCNResNet
from MethodDeepGATResNet import MethodDeepGATResNet
from MethodDeepLoopyResNet import MethodDeepLoopyResNet
from ResultSaving import ResultSaving
from SettingCV import SettingCV
from EvaluateAcc import EvaluateAcc
import numpy as np
import torch.nn as nn
import torch

/home/luis/anaconda3/envs/TFM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carga de datasets

Se cargan los datasets citados en el paper (_Cora_, _Citeseet_ y _Pubmed_), inicializando además el número de clases correspondiente para cada uno.

Ejecutaremos el dataset **Cora**, que está configurado por defecto en el código de ejemplo:

In [3]:
#---- 'cora' , 'citeseer', 'pubmed' ----

dataset_name = 'cora'

if dataset_name == 'cora':
    nclass = 7
    nfeature = 1433
elif dataset_name == 'citeseer':
    nclass = 6
    nfeature = 3703
elif dataset_name == 'pubmed':
    nclass = 3
    nfeature = 500

# Ejecución

Se testan cada uno de los modelos citados en el Paper con los distintos residuos implementados para distintas profundidades.

Ejecutaremos las siguientes celdas para comprobar los resultados.

En cada uno de los experimentos se ha decrementado el número de epochs para evitar que el kernel se interrumpa debido a problemas de memoria.

## Graph Convolutional Network

Para comprender cómo se implementan los residuos en GCN, accederemos a la clase `MethodDeepGCNResNet.py`, donde se han realizado anotaciones acerca del comportamiento de los residuos.

Originalmente el experimento estaba establecido en `1000 epochs`, se ha decrementado a `500`.

In [3]:
#---- Deep GCN ResNet baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:

            acc_test_list = []
            print('Method: GCN, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            for iter in range(1):
                #---- parameter section -------------------------------
                lr = 0.01
                epoch = 501
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.5
                nhid = 16
                #------------------------------------------------------

                #---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = 'GCN'

                method_obj = MethodDeepGCNResNet(nfeature, nhid, nclass, dropout, seed, depth)
                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepGCNResNet_' + dataset_name + '_' + residual_type + '_depth_' + str(depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                #------------------------------------------------------

                #---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                #------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')

Method: GCN, dataset: cora, depth: 1, residual type: naive
Start
Loading <class 'dataset.dataset'> dataset...
(5429, 2)
Epoch: 0001 loss_train: 1.9438 acc_train: 0.1714 loss_val: 1.9493 acc_val: 0.1433 loss_test: 1.9430 acc_test: 0.1710 time: 0.0436s
Epoch: 0051 loss_train: 1.3568 acc_train: 0.9786 loss_val: 1.6905 acc_val: 0.5900 loss_test: 1.7221 acc_test: 0.5250 time: 0.0131s
Epoch: 0101 loss_train: 1.1642 acc_train: 0.9857 loss_val: 1.6010 acc_val: 0.6200 loss_test: 1.6398 acc_test: 0.5590 time: 0.0099s
Epoch: 0151 loss_train: 1.0842 acc_train: 0.9857 loss_val: 1.5571 acc_val: 0.6233 loss_test: 1.5986 acc_test: 0.5810 time: 0.0112s
Epoch: 0201 loss_train: 1.0421 acc_train: 0.9857 loss_val: 1.5303 acc_val: 0.6300 loss_test: 1.5729 acc_test: 0.6010 time: 0.0136s
Epoch: 0251 loss_train: 1.0168 acc_train: 0.9857 loss_val: 1.5120 acc_val: 0.6133 loss_test: 1.5550 acc_test: 0.6150 time: 0.0107s
Epoch: 0301 loss_train: 1.0003 acc_train: 0.9929 loss_val: 1.4987 acc_val: 0.6233 loss_test: 1

## Graph Attention Network

Originalmente el experimento estaba establecido en `500 epochs`, se ha decrementado a `50`.

No es posible entrenar el modelo con GPU debido a  [GResNet issues](https://github.com/anonymous-sourcecode/GResNet/issues/1)

In [ ]:
#---- Deep Sparse GAT baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:

            cuda_tag = False
            print('Method: GAT, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            method_type = 'GAT'
            acc_test_list = []
            for iter in range(1):
                # ---- parameter section -------------------------------
                lr = 0.005
                epoch = 51
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.6
                nhid = 8
                nb_heads = 8
                alpha = 0.2
                patience = 100
                # ------------------------------------------------------

                # ---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = method_type

                method_obj = MethodDeepGATResNet(nfeature, nhid, nclass, dropout, seed, alpha, nb_heads, depth, cuda_tag)

                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.weight_decay = weight_decay
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepGATResNet_' + dataset_name + '_' + residual_type + '_depth_' + str(
                    depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                # ------------------------------------------------------

                # ---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                # ------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')

Method: GAT, dataset: cora, depth: 1, residual type: naive
Start
Loading <class 'dataset.dataset'> dataset...
(5429, 2)
False
Epoch: 0001 loss_train: 1.9556 acc_train: 0.1286 loss_val: 1.9438 acc_val: 0.1733 loss_test: 1.9482 acc_test: 0.1260 time: 1.1390s
Epoch: 0011 loss_train: 1.8452 acc_train: 0.5214 loss_val: 1.8938 acc_val: 0.4167 loss_test: 1.9073 acc_test: 0.3480 time: 1.0155s
Epoch: 0021 loss_train: 1.7718 acc_train: 0.7214 loss_val: 1.8510 acc_val: 0.5100 loss_test: 1.8719 acc_test: 0.4510 time: 1.3064s
Epoch: 0031 loss_train: 1.7252 acc_train: 0.7643 loss_val: 1.8138 acc_val: 0.5567 loss_test: 1.8414 acc_test: 0.4830 time: 1.3690s
Epoch: 0041 loss_train: 1.6179 acc_train: 0.8000 loss_val: 1.7815 acc_val: 0.5833 loss_test: 1.8143 acc_test: 0.4990 time: 1.4236s
Epoch: 0051 loss_train: 1.6093 acc_train: 0.7714 loss_val: 1.7529 acc_val: 0.5867 loss_test: 1.7898 acc_test: 0.5200 time: 1.2957s
Optimization Finished!
Total time elapsed: 64.7435s
Testing Acc:  tensor(0.5200, dtype=t

## Deep LoopyNet

Originalmente el experimento estaba establecido en `1000 epochs`, se ha decrementado a `500`.

In [4]:
#---- Deep LoopyNet baseline method ----
if True:
    for depth in [1, 2, 3, 4, 5, 6, 7]:
        for residual_type in ['naive', 'raw', 'graph_naive', 'graph_raw']:
            cuda_tag = False

            print('Method: LoopyNet, dataset: ' + dataset_name + ', depth: ' + str(depth) + ', residual type: ' + residual_type)
            acc_test_list = []
            for iter in range(1):
                #---- parameter section -------------------------------
                lr = 0.01
                epoch = 501
                weight_decay = 5e-4
                c = 0.1
                seed = iter
                dropout = 0.5
                nhid = 16
                #------------------------------------------------------

                #---- objection initialization setction ---------------
                print('Start')

                data_obj = DatasetLoader('', '')
                data_obj.dataset_source_folder_path = './data/' + dataset_name + '/'
                data_obj.c = c
                data_obj.method_type = 'LoopyNet'

                method_obj = MethodDeepLoopyResNet(nfeature, nhid, nclass, dropout, seed, depth, cuda_tag)
                method_obj.lr = lr
                method_obj.epoch = epoch
                method_obj.residual_type = residual_type

                result_obj = ResultSaving('', '')
                result_obj.result_destination_folder_path = './result/GResNet/'
                result_obj.result_destination_file_name = 'DeepLoopyNetResNet_' + dataset_name + '_' + residual_type+'_depth_' + str(depth) + '_iter_' + str(iter)

                setting_obj = SettingCV('', '')

                evaluate_obj = EvaluateAcc('', '')
                #------------------------------------------------------

                #---- running section ---------------------------------
                setting_obj.prepare(data_obj, method_obj, result_obj, evaluate_obj)
                acc_test = setting_obj.load_run_save_evaluate()
                print('Testing Acc: ', acc_test)
                acc_test_list.append(acc_test)
                print('*******************************')
                #------------------------------------------------------
            print(acc_test_list)
            print(np.mean(acc_test_list), np.std(acc_test_list))
            print('Finished')


Method: LoopyNet, dataset: cora, depth: 1, residual type: naive
Start
Loading <class 'dataset.dataset'> dataset...
(5429, 2)
False
Epoch: 0001 loss_train: 1.9723 acc_train: 0.1643 loss_val: 1.9222 acc_val: 0.1967 loss_test: 1.9455 acc_test: 0.1800 time: 0.0703s
Epoch: 0051 loss_train: 0.6197 acc_train: 0.9929 loss_val: 1.2285 acc_val: 0.7767 loss_test: 1.3671 acc_test: 0.7130 time: 0.0155s
Epoch: 0101 loss_train: 0.4634 acc_train: 0.9929 loss_val: 1.1077 acc_val: 0.7833 loss_test: 1.2522 acc_test: 0.7400 time: 0.0166s
Epoch: 0151 loss_train: 0.4149 acc_train: 0.9929 loss_val: 1.0588 acc_val: 0.7900 loss_test: 1.1997 acc_test: 0.7520 time: 0.0154s
Epoch: 0201 loss_train: 0.3913 acc_train: 0.9929 loss_val: 1.0308 acc_val: 0.7900 loss_test: 1.1688 acc_test: 0.7580 time: 0.0177s
Epoch: 0251 loss_train: 0.3776 acc_train: 0.9929 loss_val: 1.0129 acc_val: 0.7933 loss_test: 1.1487 acc_test: 0.7650 time: 0.0147s
Epoch: 0301 loss_train: 0.3688 acc_train: 0.9929 loss_val: 1.0008 acc_val: 0.7900 l